In [ ]:
%load_ext autoreload
%autoreload 2
import sys

import pandas as pd

sys.path.append("../..")

from mlstm_kernels.utils.benchmark.plot_results import (
    plot_benchmark_result_table,
    rc_context_wrapper,
    select_columns,
)
from pathlib import Path
from plot_config import linestyle_mapping, style_dict
import pickle

In [ ]:
# # Collect all results batch size 1
# falconmamba_gen_len_1_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_14-24-09__ttft__timetofirsttoken_gencudagraph_falconmamba_v0/hf_7B_timtofirsttok__bs1_gl1_tcTrue_weightdtypebfloat16/results.csv"
# falconmamba_gen_len_100_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_14-24-09__ttft__timetofirsttoken_gencudagraph_falconmamba_v0/hf_7B_timtofirsttok__bs1_gl101_tcTrue_weightdtypebfloat16/results.csv"

# codestralmamba_gen_len_1_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_14-35-21__ttft__timetofirsttoken_gencudagraph_codestralmamba_v0/hf_7B_timtofirsttok__bs1_gl1_tcTrue_weightdtypebfloat16/results.csv"
# codestralmamba_gen_len_100_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_14-35-21__ttft__timetofirsttoken_gencudagraph_codestralmamba_v0/hf_7B_timtofirsttok__bs1_gl101_tcTrue_weightdtypebfloat16/results.csv"

# mxlstmmamba_gen_len_1_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_15-43-57__ttft__timetofirsttoken_gencudagraph_xmlstm_v0/hf_7B_timtofirsttok__bs1_gl1_tcTrue_weightdtypebfloat16/results.csv"
# mxlstmmamba_gen_len_100_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_15-43-57__ttft__timetofirsttoken_gencudagraph_xmlstm_v0/hf_7B_timtofirsttok__bs1_gl101_tcTrue_weightdtypebfloat16/results.csv"

# llama_gen_len_1_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_15-48-51__ttft__timetofirsttoken_gencudagraph_llama_v0/hf_7B_timtofirsttok__bs1_gl1_tcFalse_weightdtypebfloat16/results.csv"
# llama_gen_len_100_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-04_15-48-51__ttft__timetofirsttoken_gencudagraph_llama_v0/hf_7B_timtofirsttok__bs1_gl101_tcFalse_weightdtypebfloat16/results.csv"

# llama_static_gen_len_1_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-05_15-14-03__ttft__llama_static_cache_v0/hf_7B_timtofirsttok__bs1_gl1_tcTrue_weightdtypebfloat16/results.csv"
# llama_static_gen_len_100_file = "/home/beck/wdir/dev_repos/mlstm_kernels/outputs_kernel_benchmarks_final/2024-12-05_15-14-03__ttft__llama_static_cache_v0/hf_7B_timtofirsttok__bs1_gl101_tcTrue_weightdtypebfloat16/results.csv"

# file_dict = {
#     "falconmamba": (falconmamba_gen_len_1_file, falconmamba_gen_len_100_file),
#     "codestralmamba": (codestralmamba_gen_len_1_file, codestralmamba_gen_len_100_file),
#     "mxlstmmamba": (mxlstmmamba_gen_len_1_file, mxlstmmamba_gen_len_100_file),
#     "llama": (llama_gen_len_1_file, llama_gen_len_100_file),
#     "llama_static": (llama_static_gen_len_1_file, llama_static_gen_len_100_file),
# }

In [ ]:
def load_timetofirst_token_results(gen_len_1_file, gen_len_100_file):
    gen_len_1_df = pd.read_csv(gen_len_1_file).filter(regex=".*(prefill|R--.*).*")
    gen_len_100_df = pd.read_csv(gen_len_100_file).filter(regex=".*(prefill|R--.*).*")
    tok_per_sec_data_df = 100 / (
        (gen_len_100_df.filter(regex=".*R--.*") - gen_len_1_df.filter(regex=".*R--.*"))
        / 1000
    )
    tok_per_sec_df = pd.concat(
        [gen_len_1_df.filter(regex=".*prefill.*"), tok_per_sec_data_df], axis=1
    )
    tok_per_sec_df, gen_len_1_df, gen_len_100_df = [
        df.rename(columns={col:col+"_vllm" for col in df.columns[1:]})
        for df in [tok_per_sec_df, gen_len_1_df, gen_len_100_df]
    ]
    return tok_per_sec_df, gen_len_1_df, gen_len_100_df

In [ ]:
dataframe_dict = load_timetofirst_token_results(
    "../../outputs_kernel_benchmarks/2025-03-26_15-19-07__ttft__testvllm/hf_7B_timtofirsttok__bs1_gl1_tcFalse_weightdtypebfloat16/results.csv",
    "../../outputs_kernel_benchmarks/2025-03-26_15-19-07__ttft__testvllm/hf_7B_timtofirsttok__bs1_gl101_tcFalse_weightdtypebfloat16/results.csv"
)

In [ ]:
dataframe_dict[0]

In [ ]:
# token_per_sec_df = pd.concat([v[0] for v in dataframe_dict.values()], axis=1)
# token_per_sec_df = pd.concat(
#     [
#         token_per_sec_df.filter(regex=".*prefill.*").take([0], axis=1),
#         token_per_sec_df.filter(regex=".*R--.*"),
#     ],
#     axis=1,
# )
# token_per_sec_df

In [ ]:
# ttft_1_df = pd.concat([v[1] for v in dataframe_dict.values()], axis=1)
# ttft_1_df = pd.concat(
#     [
#         ttft_1_df.filter(regex=".*prefill.*").take([0], axis=1),
#         ttft_1_df.filter(regex=".*R--.*"),
#     ],
#     axis=1,
# )
# ttft_1_df

In [ ]:
# ttft_100_df = pd.concat([v[2] for v in dataframe_dict.values()], axis=1)
# ttft_100_df = pd.concat(
#     [
#         ttft_100_df.filter(regex=".*prefill.*").take([0], axis=1),
#         ttft_100_df.filter(regex=".*R--.*"),
#     ],
#     axis=1,
# )
# ttft_100_df

In [ ]:
# raw_data = {
#     "token_per_sec": token_per_sec_df,
#     "ttft_1": ttft_1_df,
#     "ttft_100": ttft_100_df,
# }
# with open("ttft_raw_data.p", "wb") as f:
#     pickle.dump(raw_data, f)

In [ ]:
# for k, v in raw_data.items():
#     v.to_csv(f"raw_data_ttft_{k}.csv")

In [ ]:
with open("ttft_raw_data.p", "rb") as f:
    raw_data = pickle.load(f)

ttft_1_df = raw_data["ttft_1"]
ttft_100_df = raw_data["ttft_100"]
token_per_sec_df = raw_data["token_per_sec"]

In [ ]:
ttft_1_df = pd.concat([ttft_1_df, dataframe_dict[1].filter(regex="R.*")], axis=1)
ttft_100_df = pd.concat([ttft_100_df, dataframe_dict[2].filter(regex="R.*")], axis=1)
token_per_sec_df = pd.concat([token_per_sec_df, dataframe_dict[0].filter(regex="R.*")], axis=1)

In [ ]:
token_per_sec_df

In [ ]:
ttft_1_df

In [ ]:
raw_data = {
    "token_per_sec": token_per_sec_df,
    "ttft_1": ttft_1_df,
    "ttft_100": ttft_100_df,
}
with open("ttft_raw_data_vllm.p", "wb") as f:
    pickle.dump(raw_data, f)

## Plotting the raw data

In [ ]:
fig = plot_benchmark_result_table(
    ttft_1_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    # style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Time [ms]",
    title="Time to generate 1 tokens, for varying prefill lengths",
)

In [ ]:
fig = plot_benchmark_result_table(
    ttft_100_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    # style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Time [ms]",
    title="Time to generate 100 tokens, for varying prefill lengths",
)

In [ ]:
fig = plot_benchmark_result_table(
    token_per_sec_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    # style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Tokens per second",
    title="Tokens per second during generation of 100 tokens after consuming varying prefill lengths (bs1 gl100)",
)

## Final Plots - All results

In [ ]:
selected_columns = {
    "llama3": "R--llama3__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
    "llama2": "R--llama2__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
    "falcon_mamba": "R--falcon_mamba__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
    "codestral_mamba": "R--codestral_mamba__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
    "xlstm": "R--xlstm__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False_isd-bfloat16_ed-4096_nh-8_nb-32_vs-50304_wm-fused_ck-chunkwise--triton_xl_chunk_sk-native_sequence__triton_step_fused_sk-triton_fused_cs-128_akd-bfloat16",
}
filename_suffix = ""
add_legend = True

In [ ]:
ttft_1_plot_df = select_columns(
    ttft_1_df, selected_columns, keep_col_regex=".*prefill.*"
)

In [ ]:
fig = rc_context_wrapper(
    func=plot_benchmark_result_table,
    result_df=ttft_1_plot_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Time to First Token [ms]",
    x_label="Prefill Length",
    title="",  # "Time to generate 1 tokens, for varying prefill lengths",
    figsize=(1.2 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
    filename=f"timetofirsttoken_1_tokens{filename_suffix}",
    add_legend=add_legend,
    legend_args={
        "loc": "lower center",
        "ncol": 3,
        "bbox_to_anchor": (0.0, 1.02, 1.0, 0.502),
        "frameon": False,
        "facecolor": "white",
    },
)

In [ ]:
ttft_100_plot_df = select_columns(
    ttft_100_df, selected_columns, keep_col_regex=".*prefill.*"
)

In [ ]:
fig = rc_context_wrapper(
    func=plot_benchmark_result_table,
    result_df=ttft_100_plot_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Time to First 100 Token [ms]",
    x_label="Prefill Length",
    title="",  # "Time to generate 100 tokens, for varying prefill lengths",
    figsize=(1.2 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
    filename=f"timetofirsttoken_100_tokens_prefill_length{filename_suffix}",
    add_legend=False,  # add_legend,
    legend_args={
        "loc": "lower center",
        "ncol": 3,
        "bbox_to_anchor": (0.0, 1.02, 1.0, 0.502),
        "frameon": False,
        "facecolor": "white",
    },
)

In [ ]:
token_per_sec_plot_df = select_columns(
    token_per_sec_df, selected_columns, keep_col_regex=".*prefill.*"
)

In [ ]:
fig = rc_context_wrapper(
    func=plot_benchmark_result_table,
    result_df=token_per_sec_plot_df,
    x_axis_param="prefill_length",
    # linestyle_mapping=linestyle_mapping,
    style_dict=style_dict,
    style_dict_colname_mapping_exact=False,
    y_label="Tokens per Second",
    title="",  # "Tokens per second during generation of 100 tokens after consuming varying prefill lengths (bs1 gl100)",
    x_label="Prefill Length",
    figsize=(1.6 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
    filename=f"timetofirsttoken_tokens_per_sec{filename_suffix}",
    add_legend=add_legend,
    legend_args={
        "loc": "lower center",
        "ncol": 3,
        "bbox_to_anchor": (0.0, 1.02, 1.0, 0.502),
        "frameon": False,
        "facecolor": "white",
    },
)

In [ ]:
# selected_columns = {
#     "llama3": "R--llama3__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
#     # "llama2": "R--llama2__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
#     # "falcon_mamba": "R--falcon_mamba__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
#     "codestral_mamba": "R--codestral_mamba__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False",
#     "xlstm": "R--xlstm__tcm__ampdt-bfloat16__wdt-bfloat16__ucgg-True_ucgm-False_isd-bfloat16_ed-4096_nh-8_nb-32_vs-50304_wm-fused_ck-chunkwise--triton_xl_chunk_sk-native_sequence__triton_step_fused_sk-triton_fused_cs-128_akd-bfloat16",
# }
# filename_suffix = "only_3"

In [ ]:
# ttft_1_plot_df = select_columns(
#     ttft_1_df, selected_columns, keep_col_regex=".*prefill.*"
# )

In [ ]:
# fig = rc_context_wrapper(
#     func=plot_benchmark_result_table,
#     result_df=ttft_1_plot_df,
#     x_axis_param="prefill_length",
#     # linestyle_mapping=linestyle_mapping,
#     style_dict=style_dict,
#     style_dict_colname_mapping_exact=False,
#     y_label="Time to First Token [ms]",
#     x_label="Prefill Length",
#     title="",  # "Time to generate 1 tokens, for varying prefill lengths",
#     figsize=(1.5 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
#     filename=f"timetofirsttoken_1_tokens{filename_suffix}",
#     add_legend=add_legend,
# )

In [ ]:
# ttft_100_plot_df = select_columns(
#     ttft_100_df, selected_columns, keep_col_regex=".*prefill.*"
# )

In [ ]:
# fig = rc_context_wrapper(
#     func=plot_benchmark_result_table,
#     result_df=ttft_100_plot_df,
#     x_axis_param="prefill_length",
#     # linestyle_mapping=linestyle_mapping,
#     style_dict=style_dict,
#     style_dict_colname_mapping_exact=False,
#     y_label="Time to First 100 Token [ms]",
#     x_label="Prefill Length",
#     title="",  # "Time to generate 100 tokens, for varying prefill lengths",
#     figsize=(1.5 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
#     filename=f"timetofirsttoken_100_tokens_prefill_length{filename_suffix}",
#     add_legend=add_legend,
# )

In [ ]:
# token_per_sec_plot_df = select_columns(
#     token_per_sec_df, selected_columns, keep_col_regex=".*prefill.*"
# )

In [ ]:
# fig = rc_context_wrapper(
#     func=plot_benchmark_result_table,
#     result_df=token_per_sec_plot_df,
#     x_axis_param="prefill_length",
#     # linestyle_mapping=linestyle_mapping,
#     style_dict=style_dict,
#     style_dict_colname_mapping_exact=False,
#     y_label="Tokens per Second",
#     title="",  # "Tokens per second during generation of 100 tokens after consuming varying prefill lengths (bs1 gl100)",
#     x_label="Prefill Length",
#     figsize=(1.5 * 12 * 1 / 2.54, 1.5 * 8 * 1 / 2.54),
#     filename=f"timetofirsttoken_tokens_per_sec{filename_suffix}",
#     add_legend=add_legend,
# )